In [ ]:
# Scraping 
# https://www.skiresort.info/ski-resorts/usa/

In [9]:
# Import dependencies
import pandas as pd
import requests
import pymongo
from splinter import Browser
from bs4 import BeautifulSoup as bs
from flask import Flask, render_template, redirect
from flask_pymongo import PyMongo
from webdriver_manager.chrome import ChromeDriverManager
from pprint import pprint
import json

In [10]:
# GETTING ALL NAMES AND LINKS FOR RESORTS
url = 'https://www.skiresort.info/ski-resorts/usa/'

resort_names = []
resort_links = []

for x in range(1, 5):
    request1 = requests.get(url)

    soup1 = bs(request1.text, 'html.parser')
    
    panel = soup1.find('div', class_= "panel panel-primary")
    resort_panels = panel.find_all("div", class_="panel panel-default resort-list-item resort-list-item-image--big")

    for resort in resort_panels:
        title = resort.find("a", class_="h3")
        resort_names.append(title.text)
        resort_links.append(title["href"])
        
    url = f'https://www.skiresort.info/ski-resorts/usa/page/{x+1}'


In [19]:
len(resort_names), len(resort_links)

(523, 523)

In [19]:
# GOING THROUGH RESORT PAGES AND GATHERING RESORT SLOPE LENGTHS
resort_slopes=[]
for x in resort_links:

    request1 = requests.get(x)
    soup1 = bs(request1.text, 'html.parser')
    
    if (soup1.find("a", class_="detail-links link-img shaded zero-pad-bottom chart")):
        
        stuff1 = soup1.find("a", class_="detail-links link-img shaded zero-pad-bottom chart").find("div", class_="description")
        tot_slope= stuff1.find("div").text
        slope_type_lengths = stuff1.find("div", class_="table-graph-first").find_all("td", class_="distance")
    
        splitter = tot_slope.split()
    
        splits = []
        for y in slope_type_lengths:
            split = float(y.text.split()[0])
            splits.append(split)
    
        slope_lengths={'total': float(splitter[1]), 'easy': splits[0], 'intermediate': splits[1], 'difficult': splits[2]}
        resort_slopes.append(slope_lengths)
    else:
        resort_slopes.append('N/A')


[{'total': 234.0, 'easy': 57.0, 'intermediate': 84.0, 'difficult': 93.0},
 {'total': 88.2, 'easy': 6.7, 'intermediate': 51.7, 'difficult': 29.8},
 {'total': 150.0, 'easy': 28.5, 'intermediate': 64.5, 'difficult': 57.0},
 {'total': 89.8, 'easy': 18.4, 'intermediate': 48.5, 'difficult': 22.9},
 {'total': 237.0, 'easy': 12.0, 'intermediate': 114.0, 'difficult': 111.0},
 {'total': 153.0, 'easy': 28.0, 'intermediate': 60.0, 'difficult': 65.0},
 {'total': 135.0, 'easy': 25.0, 'intermediate': 65.0, 'difficult': 45.0},
 {'total': 84.8, 'easy': 25.5, 'intermediate': 30.5, 'difficult': 28.8},
 {'total': 117.0, 'easy': 19.9, 'intermediate': 46.7, 'difficult': 50.2},
 {'total': 94.0, 'easy': 18.0, 'intermediate': 42.0, 'difficult': 34.0},
 {'total': 143.0, 'easy': 11.0, 'intermediate': 53.0, 'difficult': 79.0},
 {'total': 70.0, 'easy': 20.0, 'intermediate': 42.0, 'difficult': 8.0},
 {'total': 100.0, 'easy': 25.0, 'intermediate': 45.0, 'difficult': 30.0},
 {'total': 165.0, 'easy': 25.0, 'intermedia

In [16]:
len(resort_slopes)

525

In [6]:
# GOING THROUGH RESORT PAGES AND GATHERING RESORT PRICES
resort_prices = []
for x in resort_links:

    request1 = requests.get(x)

    soup1 = bs(request1.text, 'html.parser')
#     stuff = soup1.find_all("div", class_="detail-links")
#     price = stuff[2].find("td").text

    if (soup1.find(id="selTicketA")):
        
        price = soup1.find(id="selTicketA").text

#     print(price)
#     if price.text: 
#     resort_prices.append(price.text)
        split1 = price.split()
        split2 = float(split1[1].split(',')[0])

        resort_prices.append(split2)
    else:
        resort_prices.append('N/A')


[219.0,
 169.0,
 218.0,
 209.0,
 199.0,
 199.0,
 176.0,
 119.0,
 165.0,
 164.0,
 164.0,
 149.0,
 209.0,
 199.0,
 133.0,
 194.0,
 199.0,
 189.0,
 209.0,
 125.0,
 199.0,
 182.0,
 'N/A',
 151.0,
 99.0,
 115.0,
 170.0,
 169.0,
 'N/A',
 165.0,
 165.0,
 95.0,
 158.0,
 129.0,
 125.0,
 74.0,
 128.0,
 145.0,
 199.0,
 125.0,
 85.0,
 79.0,
 80.0,
 119.0,
 89.0,
 119.0,
 97.0,
 119.0,
 68.58,
 84.0,
 125.0,
 89.0,
 59.0,
 99.0,
 104.0,
 89.0,
 110.0,
 84.0,
 90.0,
 63.0,
 94.0,
 64.0,
 119.0,
 115.0,
 149.0,
 62.0,
 104.0,
 135.0,
 74.0,
 89.0,
 91.0,
 100.0,
 98.0,
 99.0,
 59.0,
 110.0,
 85.0,
 84.0,
 79.0,
 100.0,
 129.0,
 89.0,
 79.0,
 46.0,
 69.0,
 85.0,
 49.0,
 79.0,
 92.0,
 104.0,
 92.0,
 129.0,
 48.0,
 89.0,
 94.0,
 78.0,
 99.0,
 85.0,
 62.0,
 70.0,
 'N/A',
 84.0,
 119.0,
 58.0,
 79.0,
 94.0,
 89.0,
 80.0,
 50.0,
 70.0,
 84.0,
 96.0,
 60.0,
 'N/A',
 49.0,
 50.0,
 79.99,
 95.0,
 'N/A',
 89.0,
 95.0,
 62.0,
 79.0,
 55.0,
 72.0,
 71.0,
 98.0,
 84.0,
 82.0,
 74.0,
 109.0,
 'N/A',
 75.0,
 49.0,


In [7]:
len(resort_prices)

525

In [11]:
# GATHERING ZIP CODES THEN GEtTING COORDINATEs
resort_zips = []

for x in resort_links:
    lilzipz = []
    url= x + 'lift-operator/'
    try:
        
        request1 = requests.get(url)
        soup1 = bs(request1.text, 'html.parser')

        
        thing = soup1.find("div", class_="panel-body middle-padding")
        lilzip = thing.find("p")
        lilzipz.append(lilzip.text.split()[0])
        bigthang =  lilzip.findAll('br')
        for x in bigthang:
            if len(x.nextSibling) > 1:
                if x.nextSibling[1].isdigit():
                    zippper = x.nextSibling.split()[0]
                    lilzipz.append(zippper)

        resort_zips.append(lilzipz[-1])


       
    except:
        
        resort_zips.append('N/A')
        
        
# Cleaning ZIP codes        
clean_zips = []
for x in resort_zips:
    if len(x) > 5:
        y= x.split('-')[0]
        clean_zips.append(y)
    else:
        clean_zips.append(x)
        
# Getting COORDINates from ZIP codes    
# Opening JSON file 
f = open('us-zip-code-latitude-and-longitude.json',) 
data = json.load(f) 
f.close() 

coords = []
for x in clean_zips:
    curr_len= len(coords)
    for y in data:
        if (x==y['fields']['zip']):
            coords.append(y['geometry']['coordinates'])
    if len(coords)==curr_len:
        coords.append('N/A')


In [14]:
clean_zips

['81658',
 '81435',
 '81658',
 '93546',
 '81612',
 '80424',
 '80435',
 '05751',
 '04261',
 '96150',
 '80482',
 '05672',
 '96146',
 '80487',
 '84060',
 '59716',
 '81612',
 '80443',
 '96146',
 '84092',
 '81612',
 '83025',
 '84060',
 '81224',
 '84310',
 '12997',
 '97702',
 '96161',
 '59716',
 '84092',
 '83353',
 '83864',
 '80435',
 '97041',
 '05155',
 '98801',
 '87525',
 '89511',
 '81612',
 '95724',
 '59937',
 '87710',
 '81147',
 '05674',
 '81301',
 '04947',
 '84121',
 '05149',
 '98229',
 '05477',
 '05859',
 '84121',
 '84719',
 '95223',
 '99587',
 '03575',
 '05356',
 '59068',
 '05464',
 '59715',
 '83702',
 '95646',
 '12853',
 '95735',
 '98022',
 '99109',
 '97028',
 '84317',
 '03251',
 '88312',
 '12442',
 '83615',
 '12496',
 '86002',
 '97028',
 '83414',
 '14731',
 '98826',
 '03580',
 '55612',
 '80466',
 '80444',
 '05751',
 '98937',
 '59827',
 '81643',
 '03249',
 '93529',
 '93634',
 '95724',
 '05673',
 '59858',
 '59640',
 '95364',
 '03860',
 '03812',
 '03255',
 '84604',
 '83837',
 '83638',


In [16]:
test_links = ['https://www.skiresort.info/ski-resort/vail/', 'https://www.skiresort.info/ski-resort/telluride/', 'https://www.skiresort.info/ski-resort/beaver-creek/']

In [28]:
# ALL TOGETHER

# NAMES AND LINKS
# url = 'https://www.skiresort.info/ski-resorts/usa/'

# resort_names = []
# resort_links = []

# for x in range(1, 5):
#     request1 = requests.get(url)

#     soup1 = bs(request1.text, 'html.parser')
    
#     panel = soup1.find('div', class_= "panel panel-primary")
#     resort_panels = panel.find_all("div", class_="panel panel-default resort-list-item resort-list-item-image--big")

#     for resort in resort_panels:
#         title = resort.find("a", class_="h3")
#         resort_names.append(title.text)
#         resort_links.append(title["href"])
        
#     url = f'https://www.skiresort.info/ski-resorts/usa/page/{x+1}'

# GOING THROUGH RESORT PAGES AND GATHERING RESORT SLOPE LENGTHS AND PRICES
all_resort_info=[]

f = open('us-zip-code-latitude-and-longitude.json',) 
zipdata = json.load(f) 
f.close() 


for x in range(0, len(resort_links)):

    resort_info = {'name': resort_names[x], 'link': resort_links[x]}
# LENGTHS   
    
    request1 = requests.get(resort_links[x])
    soup1 = bs(request1.text, 'html.parser')
    
    if (soup1.find("a", class_="detail-links link-img shaded zero-pad-bottom chart")):
    
        stuff = soup1.find("a", class_="detail-links link-img shaded zero-pad-bottom chart").find("div", class_="description")
        tot_slope= stuff.find("div").text
        slope_type_lengths = stuff.find("div", class_="table-graph-first").find_all("td", class_="distance")

        splitter = tot_slope.split()

        splits = []
        for y in slope_type_lengths:
            split = float(y.text.split()[0])
            splits.append(split)

        slope_lengths={'total': float(splitter[1]), 'easy': splits[0], 'intermediate': splits[1], 'difficult': splits[2]}

        resort_info["slopes"] = slope_lengths 
    else:
        resort_info["slopes"] = 'N/A' 

    
    
# PRICES

    if (soup1.find(id="selTicketA")):
        
        price = soup1.find(id="selTicketA").text
        
        split1 = price.split()
        split2 = float(split1[1].split(',')[0])

        resort_info["price"]= split2
        
    else:
        
        resort_info["price"]='N/A'
    
    
        
#GETTING CLOSEST TOWNS

    
    if (soup1.find("ul", class_="detail-overview-citylist")):
        
        stuff1 = soup1.find("ul", class_="detail-overview-citylist")
        town = stuff1.find("li").find("a").text

        resort_info['closest_town']= town
    
    else:
        resort_info['closest_town']= 'N/A'

        
#GETTING Regions


    reg = soup1.find(id="main-content")
    
    try:
        reg1 = reg.find_all("p")[1].find("a").text

        resort_info['region']= reg1
    
    except:
        resort_info['region']= 'N/A'
        
    
    
# GETTING ZIPS AND COORDINATES
     
   
    lilzipz = []
    url1= resort_links[x] + 'lift-operator/'
    try:
        
        request2 = requests.get(url1)
        soup2 = bs(request2.text, 'html.parser')

        
        thing = soup2.find("div", class_="panel-body middle-padding")
        lilzip = thing.find("p")
        lilzipz.append(lilzip.text.split()[0])
        bigthang =  lilzip.findAll('br')
        for x in bigthang:
            if len(x.nextSibling) > 1:
                if x.nextSibling[1].isdigit():
                    zippper = x.nextSibling.split()[0]
                    lilzipz.append(zippper)

        resort_zip = lilzipz[-1]
 
    except:
        
        resort_zip = 'N/A'
    
    
    resort_info['zip'] = resort_zip

    if len(resort_zip) > 5:
        y= resort_zip.split('-')[0]
        clean_zip = y
    else:
        clean_zip = resort_zip
    

    for y in zipdata:
         if (clean_zip==y['fields']['zip']):
            resort_info["coordinates"]= y['geometry']['coordinates']
    if 'coordinates' not in resort_info:
        resort_info['coordinates'] = 'N/A'
           

    all_resort_info.append(resort_info)

In [29]:
all_resort_info

[{'name': ' Vail ',
  'link': 'https://www.skiresort.info/ski-resort/vail/',
  'slopes': {'total': 234.0,
   'easy': 57.0,
   'intermediate': 84.0,
   'difficult': 93.0},
  'price': 219.0,
  'closest_town': 'Vail',
  'region': 'Colorado',
  'zip': '81658',
  'coordinates': [-106.497314, 39.624171]},
 {'name': ' Telluride ',
  'link': 'https://www.skiresort.info/ski-resort/telluride/',
  'slopes': {'total': 88.2,
   'easy': 6.7,
   'intermediate': 51.7,
   'difficult': 29.8},
  'price': 169.0,
  'closest_town': 'Mountain Village',
  'region': 'Colorado',
  'zip': '81435',
  'coordinates': [-107.88874, 37.932874]},
 {'name': ' Beaver Creek ',
  'link': 'https://www.skiresort.info/ski-resort/beaver-creek/',
  'slopes': {'total': 150.0,
   'easy': 28.5,
   'intermediate': 64.5,
   'difficult': 57.0},
  'price': 218.0,
  'closest_town': 'Beaver Creek Village',
  'region': 'Colorado',
  'zip': '81658',
  'coordinates': [-106.497314, 39.624171]},
 {'name': ' Mammoth Mountain ',
  'link': 'htt

In [30]:
len(all_resort_info)

523

In [31]:
all_resort_info_pd = pd.DataFrame(all_resort_info)


In [32]:
total_len = []
easy_len = []
intermediate_len = []
difficult_len = []

for i in range(len(all_resort_info)):
    if all_resort_info[i]['slopes'] != 'N/A':
        total_len.append(all_resort_info[i]['slopes']['total'])
        easy_len.append(all_resort_info[i]['slopes']['easy'])
        intermediate_len.append(all_resort_info[i]['slopes']['intermediate'])
        difficult_len.append(all_resort_info[i]['slopes']['difficult'])
    else :
        total_len.append(None)
        easy_len.append(None)
        intermediate_len.append(None)
        difficult_len.append(None)

In [33]:
all_resort_info_pd['total_len'] = total_len
all_resort_info_pd['easy_len'] = easy_len
all_resort_info_pd['intermediate_len'] = intermediate_len
all_resort_info_pd['difficult_len'] = difficult_len
all_resort_info_pd.head()

,name,link,slopes,price,closest_town,region,zip,coordinates,total_len,easy_len,intermediate_len,difficult_len
0,Vail,https://www.skiresort.info/ski-resort/vail/,"{'total': 234.0, 'easy': 57.0, 'intermediate':...",219,Vail,Colorado,81658,"[-106.497314, 39.624171]",234.0,57.0,84.0,93.0
1,Telluride,https://www.skiresort.info/ski-resort/telluride/,"{'total': 88.2, 'easy': 6.7, 'intermediate': 5...",169,Mountain Village,Colorado,81435,"[-107.88874, 37.932874]",88.2,6.7,51.7,29.8
2,Beaver Creek,https://www.skiresort.info/ski-resort/beaver-c...,"{'total': 150.0, 'easy': 28.5, 'intermediate':...",218,Beaver Creek Village,Colorado,81658,"[-106.497314, 39.624171]",150.0,28.5,64.5,57.0
3,Mammoth Mountain,https://www.skiresort.info/ski-resort/mammoth-...,"{'total': 89.8, 'easy': 18.4, 'intermediate': ...",209,Mammoth Lakes,Mammoth Lakes,93546,"[-118.86279, 37.621587]",89.8,18.4,48.5,22.9
4,Snowmass,https://www.skiresort.info/ski-resort/snowmass/,"{'total': 237.0, 'easy': 12.0, 'intermediate':...",199,Snowmass Village,Aspen Snowmass,81612,"[-106.882774, 39.223416]",237.0,12.0,114.0,111.0


In [34]:

all_resort_info_pd = all_resort_info_pd.drop(columns = ['slopes'])
all_resort_info_pd.head()

,name,link,price,closest_town,region,zip,coordinates,total_len,easy_len,intermediate_len,difficult_len
0,Vail,https://www.skiresort.info/ski-resort/vail/,219,Vail,Colorado,81658,"[-106.497314, 39.624171]",234.0,57.0,84.0,93.0
1,Telluride,https://www.skiresort.info/ski-resort/telluride/,169,Mountain Village,Colorado,81435,"[-107.88874, 37.932874]",88.2,6.7,51.7,29.8
2,Beaver Creek,https://www.skiresort.info/ski-resort/beaver-c...,218,Beaver Creek Village,Colorado,81658,"[-106.497314, 39.624171]",150.0,28.5,64.5,57.0
3,Mammoth Mountain,https://www.skiresort.info/ski-resort/mammoth-...,209,Mammoth Lakes,Mammoth Lakes,93546,"[-118.86279, 37.621587]",89.8,18.4,48.5,22.9
4,Snowmass,https://www.skiresort.info/ski-resort/snowmass/,199,Snowmass Village,Aspen Snowmass,81612,"[-106.882774, 39.223416]",237.0,12.0,114.0,111.0


In [35]:
all_resort_info_pd.to_csv("all_resorts.csv")
